Import libraries

In [21]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer





Load data and investigate format

In [22]:
pd.set_option('display.max_columns', None)

data = pd.read_csv('cover_data.csv')
print(data.head())
print(data.dtypes)


   Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0       2596      51      3                               258   
1       2590      56      2                               212   
2       2804     139      9                               268   
3       2785     155     18                               242   
4       2595      45      2                               153   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                               0                              510   
1                              -6                              390   
2                              65                             3180   
3                             118                             3090   
4                              -1                              391   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0            221             232            148   
1            220             235            151   
2            234             238   

Split the data into features and labels

Split the data into test and training sets

Standardize and normalize the data so that the continuous features are in a range between 0 and 1

In [24]:
features = data.iloc[:, :-1]
labels = data.iloc[:, -1]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=4)
numeric_features = features.iloc[:,0:10]
numeric_columns = numeric_features.columns
print(numeric_columns)
ct = ColumnTransformer([('numeric', StandardScaler(), numeric_columns)], remainder='passthrough')
standard.fit(train_features)
train_features_scaled = ct.fit_transform(train_features)
test_features_scaled = ct.transform(test_features)

train_df = pd.DataFrame(train_features_scaled)
print(train_df)


Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points'],
      dtype='object')
              0         1         2         3         4         5         6   \
0       0.312707  0.619846 -0.815006  2.963868  1.959906  2.120101 -0.117131   
1       0.462580 -1.372209 -0.815006  0.288065 -0.351184 -1.196002 -0.117131   
2      -0.761383  1.566743  0.119103 -0.412629  0.179511 -0.595463 -1.088018   
3       0.587474  1.307686  1.720433  0.283362  0.966993  0.256316 -2.843083   
4       1.236924  1.504212 -0.281229 -0.168092 -0.282707 -0.690319 -0.863967   
...          ...       ...       ...       ...       ...       ...       ...   
435754  0.865810  1.021831  0.119103 -0.549006 -0.334064  0.180046 -1.274727   
435755  0.323412 -1.309678  0.519436  1.637723 -0.950355  1.107453 -0.602575   
43

Build a model